In [ ]:
import nivapy3 as nivapy
import pandas as pd

In [ ]:
# Connect to db
ora_eng = nivapy.da.connect(src="nivabase")

In [ ]:
# Group point inputs by regine
sql = (
    "SELECT a.regine, "
    "  a.type, "
    "  c.year, "
    "  b.name, "
    "  b.unit, "
    "  SUM(c.value * d.factor) as value "
    "FROM RESA2.RID_PUNKTKILDER a, "
    "RESA2.RID_PUNKTKILDER_OUTPAR_DEF b, "
    "RESA2.RID_PUNKTKILDER_INPAR_VALUES c, "
    "RESA2.RID_PUNKTKILDER_INP_OUTP d "
    "WHERE a.anlegg_nr = c.anlegg_nr "
    "AND d.in_pid = c.inp_par_id "
    "AND d.out_pid = b.out_pid "
    "GROUP BY a.regine, a.type, c.year, b.name, b.unit "
    "ORDER BY a.regine, a.type, c.year, b.name"
)
df = pd.read_sql(sql, ora_eng)

# Just data of interest
par_list = ["As", "Cd", "Cr", "Cu", "Hg", "Ni", "Pb", "Zn"]
df.dropna(subset=["regine"], inplace=True)
df = df.query("name in @par_list")

# Save
pt_csv = r"../data/metals/point_discharges/regine_pt_dis_metals.csv"
df.to_csv(pt_csv, index=False)